### Project 7 

In this project, you will implement the the clustering techniques that you've learned this week. 

#### Step 1: Load the python libraries that you will need for this project 

In [319]:
import pandas as pd 
import matplotlib as plt
import numpy as np
import sklearn as sk 
import sqlalchemy
import patsy
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.base import TransformerMixin

%matplotlib inline

#### Step 2: Examine your data 

In [185]:
df_raw = pd.read_csv("../assets/airport_cancellations.csv")
df = df_raw.dropna() 
print df.head()

  Airport    Year  Departure Cancellations  Arrival Cancellations  \
0     ABQ  2004.0                    242.0                  235.0   
1     ABQ  2005.0                    221.0                  190.0   
2     ABQ  2006.0                    392.0                  329.0   
3     ABQ  2007.0                    366.0                  304.0   
4     ABQ  2008.0                    333.0                  300.0   

   Departure Diversions  Arrival Diversions  
0                  71.0                46.0  
1                  61.0                33.0  
2                  71.0               124.0  
3                 107.0                45.0  
4                  79.0                42.0  


In [186]:
df_raw2 = pd.read_csv("../assets/airports.csv")
df2 = df_raw2.dropna() 
print df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3498 entries, 0 to 5163
Data columns (total 13 columns):
Key                        3498 non-null float64
LocID                      3498 non-null object
AP_NAME                    3498 non-null object
ALIAS                      3498 non-null object
Facility Type              3498 non-null object
FAA REGION                 3498 non-null object
COUNTY                     3498 non-null object
CITY                       3498 non-null object
STATE                      3498 non-null object
AP Type                    3498 non-null object
Latitude                   3498 non-null float64
Longitude                  3498 non-null float64
Boundary Data Available    3498 non-null object
dtypes: float64(3), object(10)
memory usage: 382.6+ KB
None


### Intro: Write a problem statement / aim for this project

We want to understand the behavior of flight cancellations
Answer: 

### Part 1: Create a PostgreSQL database 

#### 1. Let's create a database where we can house our airport data

In [190]:
user = !id -un
print user
!rm /usr/local/var/postgres/postmaster.pid
!brew services restart postgresql
engine = sqlalchemy.create_engine('postgresql://' + user[0] + '@localhost:5432/postgres')

['alexandertam']
Stopping `postgresql`... (might take a while)
==> Successfully stopped `postgresql` (label: homebrew.mxcl.postgresql)
==> Successfully started `postgresql` (label: homebrew.mxcl.postgresql)


In [191]:
df.columns = map(str.lower, df.columns)
df2.columns = map(str.lower, df2.columns)

In [192]:
df.to_sql("airport_cancellations",con = engine, if_exists = "replace")

In [193]:
df2.to_sql("airports", con = engine, if_exists = "replace")

In [194]:
joined = pd.read_sql("SELECT * FROM airports as a JOIN airport_cancellations as ac ON a.locid=ac.airport;",con = engine)

In [195]:
!brew services stop postgresql

Stopping `postgresql`... (might take a while)
==> Successfully stopped `postgresql` (label: homebrew.mxcl.postgresql)


#### 1.2 What are the risks and assumptions of our data? 

### Part 2: Exploratory Data Analysis

#### 2.1 Plot and Describe the Data

In [196]:
joined["ap type"].unique()
joined = joined.dropna()
joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 805 entries, 0 to 804
Data columns (total 21 columns):
index                      805 non-null int64
key                        805 non-null float64
locid                      805 non-null object
ap_name                    805 non-null object
alias                      805 non-null object
facility type              805 non-null object
faa region                 805 non-null object
county                     805 non-null object
city                       805 non-null object
state                      805 non-null object
ap type                    805 non-null object
latitude                   805 non-null float64
longitude                  805 non-null float64
boundary data available    805 non-null object
index                      805 non-null int64
airport                    805 non-null object
year                       805 non-null float64
departure cancellations    805 non-null float64
arrival cancellations      805 non-null float6

In [197]:
joined.head()

,index,key,locid,ap_name,alias,facility type,faa region,county,city,state,...,latitude,longitude,boundary data available,index,airport,year,departure cancellations,arrival cancellations,departure diversions,arrival diversions
0,1223,2348.0,ABQ,ALBUQUERQUE INTL SUNPORT,Albuquerque Int'l Sunport,Airport,ASW,BERNALILLO,ALBUQUERQUE,NM,...,35.040194,-106.609194,Yes,0,ABQ,2004.0,242.0,235.0,71.0,46.0
1,1223,2348.0,ABQ,ALBUQUERQUE INTL SUNPORT,Albuquerque Int'l Sunport,Airport,ASW,BERNALILLO,ALBUQUERQUE,NM,...,35.040194,-106.609194,Yes,1,ABQ,2005.0,221.0,190.0,61.0,33.0
2,1223,2348.0,ABQ,ALBUQUERQUE INTL SUNPORT,Albuquerque Int'l Sunport,Airport,ASW,BERNALILLO,ALBUQUERQUE,NM,...,35.040194,-106.609194,Yes,2,ABQ,2006.0,392.0,329.0,71.0,124.0
3,1223,2348.0,ABQ,ALBUQUERQUE INTL SUNPORT,Albuquerque Int'l Sunport,Airport,ASW,BERNALILLO,ALBUQUERQUE,NM,...,35.040194,-106.609194,Yes,3,ABQ,2007.0,366.0,304.0,107.0,45.0
4,1223,2348.0,ABQ,ALBUQUERQUE INTL SUNPORT,Albuquerque Int'l Sunport,Airport,ASW,BERNALILLO,ALBUQUERQUE,NM,...,35.040194,-106.609194,Yes,4,ABQ,2008.0,333.0,300.0,79.0,42.0


#### Are there any unique values? 

### Part 3: Data Mining

#### 3.1 Format and Clean the Data

In [316]:
class MultiColumnLabelEncoder(TransformerMixin):
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = pre.LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = pre.LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [287]:
#test MultiColumnLabelEncoder class works

#mcle = MultiColumnLabelEncoder(columns= columnsToLabelEncode)
#mcle.fit_transform(joined)

In [315]:
class MultiColumnMinMaxScaler(TransformerMixin):
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = pre.MinMaxScaler().fit_transform(output[col].reshape(-1,1))
        else:
            for colname,col in output.iteritems():
                output[colname] = pre.MinMaxScaler().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [289]:
#test MultiColumnMinMaxScaler class works
#mcmms = MultiColumnMinMaxScaler(columns= columnsToScale)
#mcmms.fit_transform(joined).head()

In [311]:
columnsToLabelEncode = ["ap type","ap_name","alias","facility type","county","city","state","airport","faa region"]
columnsToRemove = ["locid","index","key","boundary data available"]
columnsToScale = ["year"]

In [312]:
joined = joined[[x for x in joined.columns.values.tolist() if x not in columnsToRemove]]
joined.head()

,ap_name,alias,facility type,faa region,county,city,state,ap type,latitude,longitude,airport,year,departure cancellations,arrival cancellations,departure diversions,arrival diversions
0,ALBUQUERQUE INTL SUNPORT,Albuquerque Int'l Sunport,Airport,ASW,BERNALILLO,ALBUQUERQUE,NM,Federalized/Commercial,35.040194,-106.609194,ABQ,2004.0,242.0,235.0,71.0,46.0
1,ALBUQUERQUE INTL SUNPORT,Albuquerque Int'l Sunport,Airport,ASW,BERNALILLO,ALBUQUERQUE,NM,Federalized/Commercial,35.040194,-106.609194,ABQ,2005.0,221.0,190.0,61.0,33.0
2,ALBUQUERQUE INTL SUNPORT,Albuquerque Int'l Sunport,Airport,ASW,BERNALILLO,ALBUQUERQUE,NM,Federalized/Commercial,35.040194,-106.609194,ABQ,2006.0,392.0,329.0,71.0,124.0
3,ALBUQUERQUE INTL SUNPORT,Albuquerque Int'l Sunport,Airport,ASW,BERNALILLO,ALBUQUERQUE,NM,Federalized/Commercial,35.040194,-106.609194,ABQ,2007.0,366.0,304.0,107.0,45.0
4,ALBUQUERQUE INTL SUNPORT,Albuquerque Int'l Sunport,Airport,ASW,BERNALILLO,ALBUQUERQUE,NM,Federalized/Commercial,35.040194,-106.609194,ABQ,2008.0,333.0,300.0,79.0,42.0


In [320]:
pipe = make_pipeline(
    MultiColumnLabelEncoder(columns = columnsToLabelEncode),
    MultiColumnMinMaxScaler(columns = columnsToScale),
    PCA())
result = pipe.fit_transform(joined)

result = pd.DataFrame(result)
result.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1527.112390,-24.298344,-16.965760,25.215039,-77.213215,-13.866005,-13.712336,5.539652,2.397985,8.824875,-2.478856,-1.742636,-1.286847,-0.505033,-0.112863,-7.559635e-16
1,1575.592222,-38.936172,-5.425608,19.188929,-77.746703,-14.614383,-13.661812,5.494073,2.702562,8.862859,-2.544194,-1.710426,-1.323098,-0.402181,-0.115123,-1.383558e-16
2,1351.511271,18.856052,51.357751,31.521135,-74.848603,-16.168743,-14.751531,6.116467,2.516117,7.659689,-1.874031,-1.863606,-1.407831,-0.298885,-0.105246,1.087320e-14
3,1390.855103,-24.991287,3.825915,-24.025835,-78.487402,-16.259611,-12.531668,5.621942,3.609829,9.308564,-2.390805,-1.520696,-1.297151,-0.203701,-0.105833,-1.157476e-16
4,1417.803322,-36.336851,-4.015473,7.253555,-77.670980,-15.123426,-13.329647,5.553801,2.920616,9.048486,-2.548828,-1.650853,-1.311670,-0.101662,-0.109330,-1.143931e-16


#### 3.2 Create Dummy Variables

In [183]:
# def CreateDummyVariables(df, categories):
#     categoryList = categories
#     for category in categoryList:
#         series = df[category]
#         dummies = pd.get_dummies(series, prefix=category)
#         df = pd.concat([df, dummies], axis=1)
#     return df

# joined = CreateDummyVariables(joined,['airport'])
# joined

### Part 4: Define the Data

#### 4.1 Confirm that the dataset has a normal distribution. How can you tell? 

In [ ]:
joined["departure cancellations"].plot.density()

In [ ]:
joined["departure diversions"].plot.density()

In [ ]:
joined["arrival cancellations"].plot.density()

In [ ]:
joined["arrival diversions"].plot.density()

#### 4.2 Find correlations in the data

In [ ]:
sns.heatmap(joined.corr())

#### 4.3 What is the value of understanding correlations before PCA? 

Answer: 

#### 4.4 Validate your findings using statistical analysis

#### 4.5 How can you improve your overall analysis? 

Answer: 

### Part 5: Perform a PCA and Present Findings

#### 5.1 Conduct the PCA

In [ ]:
# Create a clean data frame 
ap1 = ap[['airport','year','departure cancellations','arrival cancellations']]
print ap1.head()

#### 5.2 Write an analysis plan of your findings 

Create a writeup on the interpretation of findings including an executive summary with conclusions and next steps

### Part 6: Copy your Database to AWS 

Make sure to properly document all of the features of your dataset

### Bonus: Create a 3-Dimensional Plot of your new dataset with PCA applied